In [ ]:
!wget -nc https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh

!bash Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local

import sys
sys.path.append('/usr/local/lib/python3.8/site-packages')

!conda create -y -n py38 python=3.8

!source /usr/local/bin/activate py38 && python --version

!source /usr/local/bin/activate py38 && conda install -y pip
!source /usr/local/bin/activate py38 && pip install torch==2.0.1 transformers==4.36.2 numpy==1.24.4

!git clone https://github.com/deepseek-ai/DeepSeek-VL.git
!source /usr/local/bin/activate py38 && pip install -e ./DeepSeek-VL

--2025-05-11 04:34:40--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.32.241, 104.16.191.158, 2606:4700::6810:bf9e, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.32.241|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 155472915 (148M) [application/octet-stream]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>] 148.27M   289MB/s    in 0.5s    

2025-05-11 04:34:40 (289 MB/s) - ‘Miniconda3-latest-Linux-x86_64.sh’ saved [155472915/155472915]

PREFIX=/usr/local
Unpacking payload ...
entry_point.py:256: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
entry_point.py:256: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the fi

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Aniline.png to Aniline.png
Saving Benzaldehyde.png to Benzaldehyde.png
Saving Benzene.png to Benzene.png
Saving Benzoic_acid.png to Benzoic_acid.png
Saving Caffeine.png to Caffeine.png
Saving Morphine.png to Morphine.png
Saving Nitrobenzene.png to Nitrobenzene.png
Saving Ozone.png to Ozone.png
Saving Phenol.png to Phenol.png
Saving Pyridine-full.png to Pyridine-full.png
Saving Pyrrole-full.png to Pyrrole-full.png
Saving Pyrrole-numbered.png to Pyrrole-numbered.png
Saving Salicylic-acid.png to Salicylic-acid.png
Saving Toluene.png to Toluene.png


In [ ]:
!ls -lh /content

total 149M
-rw-r--r-- 1 root root  20K May 11 04:39 Aniline.png
-rw-r--r-- 1 root root  29K May 11 04:39 Benzaldehyde.png
-rw-r--r-- 1 root root  58K May 11 04:39 Benzene.png
-rw-r--r-- 1 root root  30K May 11 04:39 Benzoic_acid.png
-rw-r--r-- 1 root root  26K May 11 04:39 Caffeine.png
drwxr-xr-x 7 root root 4.0K May 11 04:36 DeepSeek-VL
-rw-r--r-- 1 root root 149M Apr 30 20:14 Miniconda3-latest-Linux-x86_64.sh
-rw-r--r-- 1 root root  25K May 11 04:39 Morphine.png
-rw-r--r-- 1 root root  24K May 11 04:39 Nitrobenzene.png
-rw-r--r-- 1 root root  29K May 11 04:39 Ozone.png
-rw-r--r-- 1 root root 6.6K May 11 04:39 Phenol.png
-rw-r--r-- 1 root root  25K May 11 04:39 Pyridine-full.png
-rw-r--r-- 1 root root  24K May 11 04:39 Pyrrole-full.png
-rw-r--r-- 1 root root  24K May 11 04:39 Pyrrole-numbered.png
-rw-r--r-- 1 root root  28K May 11 04:39 Salicylic-acid.png
drwxr-xr-x 1 root root 4.0K May  8 13:38 sample_data
-rw-r--r-- 1 root root  23K May 11 04:39 Toluene.png


In [ ]:
with open("task1-deepseek.py", "w") as f:
    f.write("""
from transformers import AutoTokenizer, AutoProcessor
from deepseek_vl.models import VLChatProcessor, MultiModalityCausalLM
from deepseek_vl.utils.io import load_pil_images
import torch
from PIL import Image
import os
import csv

# ----- 1. Setup Model -----
model_id = "deepseek-ai/deepseek-vl-7b-chat"
device = "cuda" if torch.cuda.is_available() else "cpu"

model = MultiModalityCausalLM.from_pretrained(model_id, trust_remote_code=True).to(torch.bfloat16).cuda().eval()
vl_chat_processor = VLChatProcessor.from_pretrained(model_id)
tokenizer = vl_chat_processor.tokenizer

# ----- 2. Define CoT Templates -----
prompts = {
    "baseline": "Which molecule is more reactive toward electrophilic aromatic substitution, the first or the second? Why?",
    "stepwise": "First, note if the ring has electron-donating substituents. Then, note if it has electron-withdrawing substituents. Finally, determine which ring is more reactive toward electrophiles.",
    "visual_first": "Observe visible features: are there -OH, -NH₂, or -NO₂ groups? Count and compare electron-donating versus electron-withdrawing groups for both molecules.",
    "explanation_first": "Consider the net electronic effect (donating vs withdrawing) based on the visible substituents. Then predict which molecule will react faster toward EAS."
}

# ----- 3. Define Image Pairs -----
dataset_folder = "/content"
image_pairs = [
    ("Aniline.png", "Nitrobenzene.png"),
    ("Benzene.png", "Toluene.png"),
    ("Benzaldehyde.png", "Benzoic_acid.png"),
    ("Pyridine-full.png", "Benzene.png"),
    ("Pyrrole-full.png", "Benzene.png"),
    ("Phenol.png", "Benzene.png"),
    ("Salicylic-acid.png", "Benzoic_acid.png"),
    ("Nitrobenzene.png", "Ozone.png"),
    ("Pyrrole-numbered.png", "Pyridine-full.png"),
    ("Morphine.png", "Caffeine.png")
]

# ----- 4. Output Setup -----
os.makedirs("outputs", exist_ok=True)
csv_file = open("outputs/Task1_deepseek_outputs.csv", "w", encoding="utf-8", newline="")
writer = csv.DictWriter(csv_file, fieldnames=["image1", "image2", "prompt_type", "prompt_text", "generated_text"])
writer.writeheader()

# ----- 5. Inference -----
for img1, img2 in image_pairs:
    img_path1 = os.path.join(dataset_folder, img1)
    img_path2 = os.path.join(dataset_folder, img2)
    for prompt_type, prompt_template in prompts.items():
        conversation = [
            {
                "role": "User",
                "content": prompt_template,
                "images": [img_path1, img_path2]
            },
            {
                "role": "Assistant",
                "content": ""
            }
        ]

        images = load_pil_images(conversation)
        prepare_inputs = vl_chat_processor(
            conversations=conversation,
            images=images,
            force_batchify=True
        ).to(model.device)

        with torch.no_grad():
            inputs_embeds = model.prepare_inputs_embeds(**prepare_inputs)
            outputs = model.language_model.generate(
                inputs_embeds=inputs_embeds,
                attention_mask=prepare_inputs.attention_mask,
                pad_token_id=tokenizer.eos_token_id,
                bos_token_id=tokenizer.bos_token_id,
                eos_token_id=tokenizer.eos_token_id,
                max_new_tokens=512,
                do_sample=False,
                use_cache=True
            )

        answer = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True)
        print(f"--- Finished {prompt_type} for {img1} and {img2}")
        writer.writerow({
            "image1": img1,
            "image2": img2,
            "prompt_type": prompt_type,
            "prompt_text": prompt_template,
            "generated_text": answer.strip()
        })

csv_file.close()
print("!!!! All pair inference completed and saved to outputs/Task1_deepseek_outputs.csv")
""")


In [ ]:
!source /usr/local/bin/activate py38 && python task1-deepseek.py

/usr/local/envs/py38/lib/python3.8/site-packages/transformers/models/auto/image_processing_auto.py:520: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
config.json: 100% 1.54k/1.54k [00:00<00:00, 257kB/s]
model.safetensors.index.json: 100% 81.1k/81.1k [00:00<00:00, 13.9MB/s]
model-00001-of-00003.safetensors:   0% 0.00/4.98G [00:00<?, ?B/s]
model-00001-of-00003.safetensors:   0% 21.0M/4.98G [00:00<00:27, 183MB/s]
model-00001-of-00003.safetensors:   1% 62.9M/4.98G [00:00<00:18, 269MB/s]
model-00001-of-00003.safetensors:   2% 94.4M/4.98G [00:00<00:22, 215MB/s]
model-00001-of-00003.safetensors:   3% 126M/4.98G [00:00<00:21, 222MB/s] 
model-00001-of-00003.safetensors:   3% 157M/4.98G [00:00<00:20, 237MB/s]
model-00001-of-00003.safetensors:   4% 18

In [ ]:
from google.colab import files
files.download('/content/outputs/Task1_deepseek_outputs.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
with open("task1-deepseek-finetune.py", "w", encoding="utf-8") as f:
    f.write("""
from transformers import AutoTokenizer
from deepseek_vl.models import VLChatProcessor, MultiModalityCausalLM
from deepseek_vl.utils.io import load_pil_images
import torch
from PIL import Image
import os
import csv

# ----- 1. Setup Model -----
model_id = "deepseek-ai/deepseek-vl-7b-chat"
device = "cuda" if torch.cuda.is_available() else "cpu"

model = MultiModalityCausalLM.from_pretrained(model_id, trust_remote_code=True).to(torch.bfloat16).cuda().eval()
vl_chat_processor = VLChatProcessor.from_pretrained(model_id)
tokenizer = vl_chat_processor.tokenizer

# ----- 2. Define Prompts (Baseline + 15 CoT prompts) -----
baseline_prompt = ["Which molecule is more reactive toward electrophilic aromatic substitution, the first or the second? Why?"]

stepwise_prompts = [
    "First, note if the ring has electron-donating substituents. Then, note if it has electron-withdrawing substituents. Finally, determine which ring is more reactive toward electrophiles.",
    "Step 1: Look at functional groups. Step 2: Predict their effects on electron density. Step 3: Predict which molecule favors EAS."
]

visual_first_prompts = [
    "Observe visible features: are there -OH, -NH₂, or -NO₂ groups? Count and compare electron-donating versus electron-withdrawing groups for both molecules.",
    "By examining only the visible substituents on the rings, conclude which molecule would favor electrophilic attack more easily."
]

explanation_first_prompts = [
    "Consider the net electronic effect (donating vs withdrawing) of all substituents. Based on that, decide which molecule is more reactive toward EAS.",
    "Electron-withdrawing groups decrease EAS reactivity. Considering this, explain which molecule's structure favors substitution more."
]

all_prompts = baseline_prompt + stepwise_prompts + visual_first_prompts + explanation_first_prompts

cot_types = (
    ["Baseline"] +
    ["Stepwise"] * len(stepwise_prompts) +
    ["Visual_first"] * len(visual_first_prompts) +
    ["Explanation_first"] * len(explanation_first_prompts)
)

# ----- 3. Define Image Pairs -----
dataset_folder = "/content"
image_pairs = [
    ("Aniline.png", "Nitrobenzene.png"),
    ("Benzene.png", "Toluene.png"),
    ("Benzaldehyde.png", "Benzoic_acid.png")
]

# ----- 4. Output Setup -----
os.makedirs("outputs", exist_ok=True)
csv_file = open("outputs/Task1_deepseek_finetune-res.csv", "w", encoding="utf-8", newline="")
writer = csv.DictWriter(csv_file, fieldnames=["image1", "image2", "prompt_type", "prompt_number", "prompt_text", "generated_text"])
writer.writeheader()

# ----- 5. Inference -----
for img1, img2 in image_pairs:
    img_path1 = os.path.join(dataset_folder, img1)
    img_path2 = os.path.join(dataset_folder, img2)
    for idx, prompt_template in enumerate(all_prompts):
        prompt_type = cot_types[idx]
        conversation = [
            {
                "role": "User",
                "content": prompt_template,
                "images": [img_path1, img_path2]
            },
            {
                "role": "Assistant",
                "content": ""
            }
        ]

        images = load_pil_images(conversation)
        prepare_inputs = vl_chat_processor(
            conversations=conversation,
            images=images,
            force_batchify=True
        ).to(model.device)

        with torch.no_grad():
            inputs_embeds = model.prepare_inputs_embeds(**prepare_inputs)
            outputs = model.language_model.generate(
                inputs_embeds=inputs_embeds,
                attention_mask=prepare_inputs.attention_mask,
                pad_token_id=tokenizer.eos_token_id,
                bos_token_id=tokenizer.bos_token_id,
                eos_token_id=tokenizer.eos_token_id,
                max_new_tokens=512,
                do_sample=False,
                use_cache=True
            )

        answer = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True)
        print(f"--- Finished {prompt_type} #{idx+1} for {img1} and {img2}")
        writer.writerow({
            "image1": img1,
            "image2": img2,
            "prompt_type": prompt_type,
            "prompt_number": idx + 1,
            "prompt_text": prompt_template,
            "generated_text": answer.strip()
        })

csv_file.close()
print("!!!! All pair inference completed and saved to outputs/Task1_deepseek_finetune-res.csv")
""")


In [ ]:
!source /usr/local/bin/activate py38 && python task1-deepseek-finetune.py

/usr/local/envs/py38/lib/python3.8/site-packages/transformers/models/auto/image_processing_auto.py:520: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
Loading checkpoint shards: 100% 3/3 [00:04<00:00,  1.35s/it]
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can 

In [ ]:
from google.colab import files
files.download('/content/outputs/Task1_deepseek_finetune-res.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>